# Imports

In [1]:
import pandas as pd 
import pandas as pd
from sqlalchemy import create_engine, inspect
from sqlalchemy.orm import Session

# read in csvs

In [2]:
df1 = pd.read_csv('Imdb_to_rt.csv')
df2 = pd.read_csv('rt_to_imdb.csv')

# add a space into Movies from imdb movies since there was originally none between the movie name and the year that it was creted so they would appear as:
# The Shawshank Redemption(1994)

In [3]:
space = []
for movie in df1['Movie']:
    x = movie.replace('(',' (')
    space.append(x)

# replace column of originally unspaced movies with now spaced movies 
# clean dataframe to only contain same columns as df2 

In [4]:
df1['Movie'] = space
df1 = df1[['Movie','Tomatometer','IMDB score']]
df1

,Movie,Tomatometer,IMDB score
0,The Shawshank Redemption (1994),91%,9.3
1,The Godfather (1972),97%,9.2
2,The Dark Knight (2008),94%,9.0
3,The Godfather: Part II (1974),96%,9.0
4,12 Angry Men (1957),100%,9.0
...,...,...,...
95,North by Northwest (1959),97%,8.3
96,Vertigo (1958),94%,8.3
97,Singin' in the Rain (1952),100%,8.3
98,Citizen Kane (1941),99%,8.3


# clean dataframe to only contain same columns as df1 

In [5]:
df2 = df2[['Tomatometer','Movie','IMDB score']]

# final merge of data into dataframe 

In [6]:
final = df1.merge(df2, how='outer')
final

,Movie,Tomatometer,IMDB score
0,The Shawshank Redemption (1994),91%,9.3
1,The Godfather (1972),97%,9.2
2,The Dark Knight (2008),94%,9.0
3,The Godfather: Part II (1974),96%,9.0
4,12 Angry Men (1957),100%,9.0
...,...,...,...
182,Top Hat (1935),100%,7.8
183,Never Rarely Sometimes Always (2020),99%,7.4
184,Pain and Glory (Dolor y gloria) (2019),96%,7.5
185,Shoplifters (Manbiki kazoku) (2018),99%,7.9


# Path to sqlite

In [7]:
mv_df1 = pd.read_csv('Loaded_1.0.csv')
database_path = ("movie.sqlite")

# Create an engine that can talk to the database

In [8]:
engine = create_engine(f"sqlite:///{database_path}")
mv_df1.to_sql(name='movie', con=engine, if_exists='append', index=False)

# Module used to connect Python with MongoDb
# create a local database through mongodb

In [9]:
import pymongo

conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

db = client.movie
client.drop_database('movie')
db = client.movie
movie_coll = db.movie_coll
mv_df1 = pd.read_csv('Loaded_1.0.csv')

for idx in mv_df1.index:
    row = mv_df1.loc[idx]
    movie_coll.insert_one(
        {
        'title': row['Movie'],
        'Tomatometer': row['Tomatometer'],
        'IMDB score': row['IMDB score'],
        }
    ) 

# This is load to Mongodb

In [10]:
movies = movie_coll.find()
for movie in movies:
    print(movie)

{'_id': ObjectId('60c454486c9859ef06cd10fe'), 'title': 'The Shawshank Redemption (1994)', 'Tomatometer': '91%', 'IMDB score': 9.3}
{'_id': ObjectId('60c454486c9859ef06cd10ff'), 'title': 'The Godfather (1972)', 'Tomatometer': '97%', 'IMDB score': 9.2}
{'_id': ObjectId('60c454486c9859ef06cd1100'), 'title': 'The Dark Knight (2008)', 'Tomatometer': '94%', 'IMDB score': 9.0}
{'_id': ObjectId('60c454486c9859ef06cd1101'), 'title': 'The Godfather: Part II (1974)', 'Tomatometer': '96%', 'IMDB score': 9.0}
{'_id': ObjectId('60c454486c9859ef06cd1102'), 'title': '12 Angry Men (1957)', 'Tomatometer': '100%', 'IMDB score': 9.0}
{'_id': ObjectId('60c454486c9859ef06cd1103'), 'title': 'The Lord of the Rings: The Return of the King (2003)', 'Tomatometer': '93%', 'IMDB score': 8.9}
{'_id': ObjectId('60c454486c9859ef06cd1104'), 'title': 'Pulp Fiction (1994)', 'Tomatometer': '92%', 'IMDB score': 8.9}
{'_id': ObjectId('60c454486c9859ef06cd1105'), 'title': "Schindler's List (1993)", 'Tomatometer': '98%', 'IM